In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
plt.style.use('fivethirtyeight')

import warnings
warnings.filterwarnings('ignore')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

%matplotlib inline
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Part1: 탐색적 데이터 분석(EDA):
1) 기능 분석.

2) 여러 기능을 고려하여 관계 또는 추세를 찾습니다.

## Part2: 기능 엔지니어링 및 데이터 정리:
1) 몇 가지 기능을 추가합니다.

2) 중복 기능 제거.

3) 모델링에 적합한 형태로 피쳐 변환.

## Part3: 예측 모델링
1) 기본 알고리즘 실행.

2) 교차 검증.

3) 앙상블.

4) 중요 특징 추출.

# Part1: Exploratory Data Analysis(EDA)<br>(탐색적 데이터 분석)

In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [3]:
data = pd.read_csv('../input/titanic/train.csv')

In [4]:
data.head()

In [5]:
data.isnull().sum()

*** Age,Cabin 및 Embarked 에는 null값이 있음**

# 생존자는 몇명인가?

* 타이타닉 영화를 본 사람은 알듯이 매우 절망적인 사고 현장. 많은 사람이 사망한 사건으로 실제로 생존자와 사망자의 비율 확인

In [6]:
# 생존률 확인하기 -> (얼마나 많은 사람이 생존했는지)
f,ax =plt.subplots(1,2,figsize =(18,8))
# Survived의 값들을 count해서 pie 그래프로 시각화
data['Survived'].value_counts().plot.pie(explode=[0,0.1],autopct='%1.1f%%',ax=ax[0],shadow=True)
ax[0].set_title('Survived')
ax[0].set_ylabel('')
## Survived의 값들을 count해서 막대 그래프로 시각화
sns.countplot('Survived', data=data, ax=ax[1])
ax[1].set_title('Survived')
plt.show()

* train data에 있던 891명의 승객 중 약 38.4%(약 342명)만 생존 
* 대략 61%의 승객이 사망한것으로 보인다

# * Feature Analysis

## Categorical(범주형) - 질적 변수
>명목형(Norminal) 변수라고도 한다.타이타닉 데이터에선 Sex, Embarked에 해당하는 데이터. 순서와는 무관하여 정렬을 할 수 없다.

## Ordinay(순서형) - 질적 변수
>명목형 변수, 예를 들어 성별은 순서의 의미가 없다고 했다. 남자가 1, 여자가 2라고 해서 1에 가중치를 더 둘 수 없는것은 상식!
그에 반해 순서형 변수는 이름답게 순서에 따라 의미를 둘 수 있다. 이 데이터에서는 Pclass데이터가 해당된다. Pclass는 1, 2, 3으로 나눠져 있는데 이는 의미를 둘 수 있다. 다른 예로서, 설문조사에서 만족, 불만족의 단계도 또한 순서형 데이터라고 할 수 있다.

## Continous(연속형) - 양적 변수
>말 그대로 연속적인 값. 키, 몸무게 등이 이에 해당한다. 이 데이터에서는 Age가 해당한다.(사실 나이가 연속형이라고도 할 수 있고, 이산형이라고도 할 수 있다.)

## Discrete(이산형) - 양적 변수
>이산형 변수는 연속형 변수와 다르게 셀 수 있는 변수이다.

# Sex-> Categorical Feature

In [7]:
data.groupby(['Sex','Survived'])['Survived'].count()

* Groupby를 통해 성별과 생존 유무에 따라 묶고 생존자 및 사망자의 수를 세어봄
* 남녀 전체 수는 일단 신경쓰지 않고, 사망자와 생존자의 수만 비교하면 여성 생존자가 더 많다

In [8]:
f,ax = plt.subplots(1,2,figsize=(18,8))
data[['Sex','Survived']].groupby(['Sex']).mean().plot.bar(ax=ax[0])
ax[0].set_title('Survived vs Sex')
sns.countplot('Sex', hue='Survived',data = data, ax=ax[1])
# hue인자에 카테고리컬 변수를 지정하면 카테고리값에 따라 색을 변화
ax[1].set_title('Sex : Survived vs Dead')
plt.show()

* 시각화 그래프로 확인해본 결과 : <br>
배에 타고 있는 남자의 수가 여자의 수보다 훨씬 많다. 여전히 구조된 여성의 수는 구조된 남성의 거의 두 배이다. 배에 타고 있는 여성의 생존율은 약 75%인 반면, 남성의 생존율은 약 18-19%이다.

# Plcass -> Ordinal Feature

* Pclass는 객실 클래스에 대한 변수이다. 3 -> 2 -> 1순으로 등급이 나눠진다.

In [9]:
pd.crosstab(data.Pclass,data.Survived,margins=True).style.background_gradient(cmap='summer_r')
# background_gradient로 heatmpas를 만들 수 있다.(matplotlib 모듈), 값의 크기에 따라 gradient 변화
# margins 파라미터로 행, 열 합 추가 가능

*  crosstab으로 변수 케이스 별 value_count가 가능하다.
*  crosstab은 범주형 변수로 되어있는 요인별로 교차분석을 하여, 행 및 열 요인 기준별로 빈도를 세어서 도수분포표, 교차표를 만들어준다.

* Pclass별 Survived의 교차분석을 시행한 결과 생존자가 가장 많은 class는 1, 사망자가 많은 클래스는 3이다.

In [10]:
f,ax = plt.subplots(1,2,figsize=(18,8))

data['Pclass'].value_counts().plot.bar(color = ['#CD7F32','#FFDF00','#D3D3D3'],ax=ax[0])

ax[0].set_title('Number Of Passengers By Pclass')
ax[0].set_ylabel('Count')

sns.countplot('Pclass', hue ="Survived", data =data,ax=ax[1])
ax[1].set_title('Pclass:Survived vs Dead')

plt.show()

* 사람들은 돈으로 모든 것을 살 수는 없다고 말합니다. 그러나 우리는 Pclass 1의 승객들이 구조하는 동안 매우 높은 우선순위를 부여받았음을 분명히 알 수 있다. Pclass 3의 승객 수는 훨씬 더 많았지만, 생존자 수는 여전히 약 25%로 매우 낮습니다.

* Pclass1의 경우 생존율이 약 63%인 반면 Pclass2의 경우 생존율이 약 48%입니다. 그래서 돈과 지위가 생존율에 영향을 미쳤던 것 같습니다


#### Sex 와 Pclass별 생존자 파악하기

In [11]:
pd.crosstab([data.Sex,data.Survived],data.Pclass,margins=True).style.background_gradient(cmap = 'summer_r')

In [12]:
sns.factorplot('Pclass',"Survived", hue = 'Sex', data=data)

plt.show()

* Pclass와 무관하게 여성이 우선시 구조되었다.
* CrossTab 와 FactorPlot를 보면 Pclass1의 여성94명 중 3명만 사망했기 때문에 Pclass1의 여성 생존율이 약95~96%임을 쉽게 추론가능
* Pclass1의 남성들도 생존율(37%)이 매우 낮음
* Pclass3의 남자들은 생존율이 13%밖에 되지 않는다<br>
즉, Pclass3의 남자들의 생존율은 더 희박하다는 의미

# Age -> Continous Feature

In [13]:
print('Oldset Passenger was of :', data['Age'].max(),'Years')
print('Youngest Passenger was of :', data['Age'].min(),'Years')
print('Average Age on the ship :', data['Age'].mean(),'Years')

* 승객들의 나이 분포를 보면 0 ~ 80살 정도이고, 평균은 대략 30살

In [14]:
# Pclass별 나이데 따른 승객들의 생존율
f,ax = plt.subplots(1,2,figsize=(18,8))

# violinplot -> boxplot처럼 분포도 볼 수 있음
sns.violinplot('Pclass','Age',hue='Survived', data =data, split=True,ax=ax[0])
ax[0].set_title('Pclass and Age vs Survived')
ax[0].set_yticks(range(0,110,10))

sns.violinplot('Sex','Age', hue='Survived',data=data,split=True,ax=ax[1])
ax[1].set_title('Sex and Age vs Survived')
ax[1].set_yticks(range(0,110,10))

plt.show()

1) 10세 미만의 아이의 수는 Pclass 1 -> 3으로 증가하면서 같이 증가하며, 대부분 생존자로 속함<br>
2) Pclass1의 20-50세 승객은 생존 확률이 높고 여성의 생존자가 많습니다./하지만 전체적으로 20-50대의 사망 승객이 많다.<br>
3) 남자의 경우 나이가 많을수록 생존확률이 줄어드는것을 알수 있다 (대략 65세 이상의 노인은 사망자가 더 많은 것을 보았을때)

* Age feature에는 177개의 null 값이 있음 때문에 NaN값을 대체하기 평균값을 넣고자 했으나, 
* 다른 연령대의 사람들이 있기 떄문에 평균 연령 값을 배정할 수 없음
* name feature의 중 이니셜( Mr, Mrs와 같은 경치)을 가지고 있음
* 따라서 각 그룹의 이니셜(Mr,Mrs)따라 평균 값을 할당

In [15]:
# 이름중에 Mr, Mrs들어가는 사람의 나이중 결측값에 평균을 대입
data['Initial'] =0
for i in data:
    data['Initial']=data.Name.str.extract('([A-Za-z]+)\.') #lets extract the Salutations
    #위 extraxt의 의미는 A-Z와 a-z의 문자중 다음 문자로 .이 따라오는 것을 추출

In [16]:
pd.crosstab(data.Initial,data.Sex).T.style.background_gradient(cmap='summer_r')
# checking the Initials with the sex

* Mlle 또는 Mme와 같이 Miss를 나타내는 잘못된 이니셜이 있다. 오타라고 생각되는 이니셜들을 수정▼(아래 코드)

In [17]:
data['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don'],['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr'],inplace=True)

In [18]:
# 이니셜별 나이의 평균을 구해서 이니셜을 가진 승객이 결측치라면 구한 평균값을 넣기
data.groupby('Initial')['Age'].mean()
#lets check the average age by Initials

## Filling NaN Ages (Age의 결측치 채우기)

In [19]:
## Assigning the NaN Values with the Ceil values of the mean ages

data.loc[(data.Age.isnull())&(data.Initial== 'Mr'),'Age']=33
data.loc[(data.Age.isnull())&(data.Initial== 'Mrs'),'Age']=36
data.loc[(data.Age.isnull())&(data.Initial== 'Master'),'Age']=5
data.loc[(data.Age.isnull())&(data.Initial== 'Miss'),'Age']=22
data.loc[(data.Age.isnull())&(data.Initial== 'Other'),'Age']=46

In [20]:
data.Age.isnull().any()
# So no null vlaues left finally
# Null 값이 있는지 확인

In [21]:
f,ax = plt.subplots(1,2,figsize=(20,10))

data[data['Survived']==0].Age.plot.hist(ax=ax[0],bins=20,edgecolor='black',color='red')
ax[0].set_title('Survived=0')
x1=list(range(0,85,5))
ax[0].set_xticks(x1)

data[data['Survived']==1].Age.plot.hist(ax=ax[1],bins=20,edgecolor='black',color='green')
ax[1].set_title('Survived=1')
x2=list(range(0,85,5))
ax[1].set_xticks(x2)

plt.show()

* 30-40 대의 승객의 사망율이 높다 즉, 최대 사망자는 30~40세 연령임
* 아이들의 생존을 우선시함 (유아[만5세 미만]가 우선 구제),(여성 및 아동 우선적 구조 정책)
* 최고령 승객은 구조됨 (80세 이상의 노인은 구조됨)

In [22]:
sns.factorplot('Pclass','Survived',col='Initial',data=data)

plt.show()

# Embatked(승선함, 항구명) -> Categorical Value

In [23]:
pd.crosstab([data.Embarked,data.Pclass],[data.Sex,data.Survived],margins=True).style.background_gradient(cmap='summer_r')

* S의 승객이 가장 많고 남성 사망자가 많다

## Chances for Survival by Port of Embarkation

In [24]:
sns.factorplot('Embarked',"Survived",data=data)
fig=plt.gcf() # figure에 접근해야할 때 사
fig.set_size_inches(5,3)

plt.show()

* C 항구의 생존 확률은 0.55 정도로 가장 높고, S 항구는 생존 확률 35%도 안되게 가장 낮다.

In [25]:
f,ax = plt.subplots(2,2,figsize=(20,15))

sns.countplot('Embarked',data=data, ax=ax[0,0])
ax[0,0].set_title('No. of Passengers Boarded') # 탑승객 수

sns.countplot('Embarked',hue='Sex',data=data, ax=ax[0,1])
ax[0,1].set_title('Male-Female Split for Embarked') #남녀별 탑승 수

sns.countplot('Embarked', hue='Survived',data=data, ax=ax[1,0])
ax[1,0].set_title('Embarked vs Survived') # 탑승 대비 생존 

sns.countplot('Embarked', hue='Pclass',data=data, ax=ax[1,1])
ax[1,1].set_title('Embarked vs Pclass') # 탑승 등급

plt.subplots_adjust(wspace=0.2, hspace=0.5)

plt.show()

* S에서 탑승한 대부분 Pclass3의 승객들이다
* S에서 탑승했을 때 생존확률이 낮다, 그 이유는 Pclass3의 대략 81%사람이 사망했기 때문
* C에서 탑승한 승객은 상당수 생존했다. 이는 운이좋은 것으로 보이며, 그 이유는 Pclass1과 Pclass2 승객의 구조때문일것으로 판단된다
* Q에는 승객의 거의 95%가 Pclass3에서 왔다

In [26]:
sns.factorplot('Pclass','Survived',hue='Sex',col='Embarked',data=data)

plt.show()

* Pclass1, Pclass2의 생존 확률은 Pclass에 관계없이 여성의 경우 생존확률이 높음
* S에서 Pclass3의 남성 여성 모두 생존확률이 매우 낮다 (돈문제인거 같다)
* Q는 Pclass3의 승객이 대부분이며 남성들의 생존 확률이 매우 희박하다

## Filling Embarked NaN (탑승 칼럼 NaN 값 채우기)

In [27]:
# Embarked의 결측치(Null value)는 최빈값으로 채우기
data["Embarked"].fillna('S',inplace=True) # S로 대체

In [28]:
data.Embarked.isnull().any() 
# Finally No NaN Values

# SiSip -> Discrete Feature

* 사람이 혼자 있는지, 가족과 함께 있는지 나타내는 feature

* Sibling = brother, sister, stepbrother, stepsister

* Spouse = husband, wife

In [29]:
pd.crosstab([data.SibSp], data.Survived).style.background_gradient(cmap='summer_r')

In [30]:
f,ax = plt.subplots(1,2,figsize=(20,8))

sns.barplot('SibSp','Survived',data=data, ax=ax[0])
ax[0].set_title('SibSp vs Survived')


sns.pointplot('SibSp','Survived',data=data,ax=ax[1])
ax[1].set_title('SibSp vs Survived')

#plt.close(2)

plt.show()

# <span style="color:red">문제 발생

![](https://mblogthumb-phinf.pstatic.net/MjAyMTA4MDVfMTUg/MDAxNjI4MTU4NTc5MzI3.be7VtT7WBiKM7YsiSlDVpc4pVJT891o37IzrRjIL8SYg.vZgOE3z4lALyXQU2egXlZdG9-hh8CPBwCwM7JKUjZpgg.PNG.fbfbf1/image.png?type=w800)

> 발생 이유 : factorplot은 figure-level이라서 자기가 figure를 하나 새로 만듦 즉, ax에 담기지 않고 새로운 figure를 만들어내기때문에 문제 발생 

# <span style="color:red">해결법 
* 참고자료 : https://stackoverflow.com/questions/54959764/seaborn-factorplot-generates-extra-empty-plots-below-actual-plot
> plt.close(2)와 plt.close(3)을 실행했지만 factorplot가 그려지지 않았음. 
>> pointplot 사용 : axes-level이라 기존의 axes에 담겨서 문제없고 똑같이 그려줌 
>>> 다른 방법으로 factorplot ->catplot로 변경됨 문서 확인하여 사용하면 됩니다

In [31]:
pd.crosstab(data.SibSp, data.Pclass).style.background_gradient(cmap="summer_r")

* 막대 그래프를 보면 혼자 온 승객의 생존율 :34.5%
* 형제 자매의 수가 증가하면 막대그래프의 수치가 감소하는데 이는 배에 가족이 있으면 나 자신을 먼저 구하는 것이 아니라 가족을 구하려고 하기때문이다 
* 하지만, 5~8인 가족의 생존율은 0% 
* 이는 Pclass 차이 때문 SibSp>3 : 모두 Pclass3임 : Pclass3(>3)의 모든 대가족이 사망

* 혼자이거나, 3명이상(자신포함X)일때 생존 확률이 낮아진다

# Parch

In [32]:
pd.crosstab(data.Parch,data.Pclass).style.background_gradient(cmap='summer_r')

In [33]:
f,ax = plt.subplots(1,2,figsize=(20,8))

sns.barplot('Parch','Survived',data=data,ax=ax[0])
ax[0].set_title('Parch vs Survived')

sns.pointplot('Parch','Survived',data=data,ax=ax[1])
ax[1].set_title('Parch vs Survived')

#plt.close(2)
plt.show()

* 위의 결과는 SibSp와 비슷하다. 부모를 태운 승객은 생존 가능성이 더 높다 
* 하지만, 부모님의 수가 높아질수록 생존율은 감소한다 
* 즉, 배에 부모가 1-3명 있는 사람은 생존율이 좋음
* 하지만, 혼자 있는 것은 생존율에 치명적이다 즉, 생존율이 낮다
* 배에 4명 이상의 부모가 있을 때, 생존율은 줄어든다
[정리]
* Parch data :1-3에 해당하는 승객은 생존율 높고, 나머지는 낮다 

# Fare -> Continous Feature

In [34]:
print('Highest Fare was :', data['Fare'].max())
print('Lowest Fare was:', data['Fare'].min())
print('Average Fare was:',data['Fare'].mean())

In [35]:
f,ax=plt.subplots(1,3,figsize=(20,8))

sns.distplot(data[data['Pclass']==1].Fare,ax=ax[0])
ax[0].set_title('Fares in Pclass 1')

sns.distplot(data[data['Pclass']==2].Fare,ax=ax[1])
ax[1].set_title('Fares in Pclass 2')

sns.distplot(data[data['Pclass']==3].Fare,ax=ax[2])
ax[2].set_title('Fares in Pclass 3')

plt.show()

## 정리

* Sex : 남성보다 여성의 생존율이 높다. 이는 구조가 여성이 우선한다

* Pclass : Pclass 1의 승객의 생존율이 높다. 반면에 3은 낮다. 여성에게는 Pclass 1이 거의 1등이고, 2등도 높다.

* Age : 5-10살의 아이들의 생존율이 높고 15-35살의 성인들이 많이 사망했다

* Embarked : Q의 사람들은 대다수가 Pclass 3. Embarked중에서 C만 생존율이 사망률보다 높다.

* Parch + SibSp : 1-2 SibSP, Spouse on board or 1-3 Parents의 생존율이 높다. 혼자오거나 대가족은 상대적으로 낮다.

# Correlation Between The Features <br>(기능간 상관관계)

In [36]:
sns.heatmap(data.corr(),annot=True,cmap='RdYlGn',linewidths=0.2)
# data.corr() -> correlation matrix
# annot = True : 각 셀에 숫자를 입력

fig = plt.gcf()
fig.set_size_inches(10,8)
plt.show()

# 알파벳이나 문자열 사이에 상관관계는 없기 때문에 수치형 변수에만 적용
# 만약 상관계수가 높다면 다중공산성 문제 발생 
# 이는 ML 모델을 학습시킬 때 성능 약화

* SibSp와 Parch의 0.41로 상관계수는 상대적으로 높은 편에 속한다


# Part2 : Feature Engineering and Data Cleaning<br>(데이터 정제)

* 모든 feature가 중요하지 않기 때문에
* 중복되어 필요없는 feature는 삭제(제거)
* 또 다른 feature를 관찰하거나 정보를 추출하여 새로운 의미 있는 데이터를 추출

# Age_band

* Age는 연속형 변수 -> ML모델(머신러닝 모델)에 문제가 발생할 가능성 있다
* 연속형 변수를 범주형 변수로 변환 필요 ( Binning or Normalization )<br>
    연령 범위를 단일 bin으로 묶거나 단일 값을 할당할 예정
* 나이(0 -80세)을 5개로 나뉘면 16세 단위로 전체 나이의 사이즈를 나눔

In [37]:
data['Age_band']=0

data.loc[data['Age']<=16,'Age_band']=0
data.loc[(data['Age']>16)&(data['Age']<=32),'Age_band']=1
data.loc[(data['Age']>32)&(data['Age']<=48),'Age_band']=2
data.loc[(data['Age']>48)&(data['Age']<=64),'Age_band']=3
data.loc[data['Age']>64,'Age_band']=4

data.head(2)

In [38]:
data['Age_band'].value_counts().to_frame().style.background_gradient(cmap='summer_r')
# checking the number of passengers in each band

In [39]:
sns.factorplot('Age_band','Survived',data=data,col='Pclass')
plt.show()

* 나이가 많아질수록 생존율이 급감하는데 이는 Pclass와 무관함

# Family_Size and Alone

* Family_size 와 Alone이라는 새로운 feature를 생성하고 분석할 수 있음
* 이 featrue는 Parch와SibSp의 합이다
* 생존율이 탑승자의 가족 규모와 관련이 있는지 확인할 수 있도록 결합된 데이터를 제공
* 혼자인 승객이 혼자인지 아닌지를 유무를 나타내는 feature를 만들기

In [40]:
data['Family_Size']=0
data['Family_Size']=data['Parch']+data['SibSp'] #Family size
data['Alone']=0
data.loc[data.Family_Size==0,'Alone']=1

In [41]:
f,ax = plt.subplots(1,2,figsize=(18,6))

sns.pointplot('Family_Size','Survived',data=data, ax=ax[0])
ax[0].set_title('Family_Size vs Survived')

sns.pointplot('Alone','Survived',data=data, ax=ax[1])
ax[1].set_title('Alone vs Survived')

#plt.close(2)
#plt.close(3)
plt.show()

* Family_size =0은 혼자 온 승객을 의미함
* Family_size =0이라면 생존 가능성이 매우 낮다 
* Family_size =4보다 클 경우도 생존 확률이 감소
* Family_size = 1,2,3 일 때가 생존율이 높다

In [42]:
sns.factorplot('Alone','Survived',data=data,hue='Sex',col='Pclass')
plt.show()

* 혼자 있는 여성이 가족이 있는 여성보다 혼자일 확률이 높은 Pclass3를 제외
* 성별이나, Pclass를 불문하고 혼자 있는 것이 생존율에 좋지 못하다는 것을 알 수 있다

# Fare_Range

* Fare feature 도 연속 데이터이기에 이산형 변수로 변환해야함 ( pandas.qcut 사용)


In [43]:
data['Fare_Range']= pd.qcut(data['Fare'],4)
# qcut : 우리가 전달한 bin에 따라 값을 분할하거나 정렬
# bin :5를 전달하면 값이 5개의 별도 bin 또는 정해진 값 범위에 동일한 간격으로 값을 정렬
data.groupby(['Fare_Range'])['Survived'].mean().to_frame().style.background_gradient(cmap='summer_r')

In [44]:
data['Fare_cat']=0
data.loc[data['Fare']<=7.91, 'Fare_cat']=0
data.loc[(data['Fare']>7.91)&(data['Fare']<=14.454),'Fare_cat']=1
data.loc[(data['Fare']>14.454)&(data['Fare']<=31),'Fare_cat']=2
data.loc[(data['Fare']>31)&(data['Fare']<=513),'Fare_cat']=3

* Fare 높을수록 생존율이 높아지는 것을 알 수 있음
* Fare_Range값을 그대로 전달할 수 없어 위의 Age_Band에서 했던 것과 같이 singleton values로 변환

In [45]:
sns.factorplot('Fare_cat','Survived',data=data,hue='Sex')
plt.show()

* Fare_cat가 증가할수록 생존확률도 증가함
* Fare feature는 Sex와 함께 모델링하면 유효한 data를 유추할 수 있을것 같음

# Converting String Values into Numeric <br> 문자열값을 숫자로 변환

* 머신러닝 모델에는 문자열을 전달할 수 없기 때문에 Sex, Embakred 등 feature는 숫자 값으로 변화해야함

* 머신러닝에 문자열이 있으면 학습율이 떨어져서 하는것임. 

In [46]:
data['Sex'].replace(['male','female'],[0,1],inplace=True)
data['Embarked'].replace(['S','C','Q'],[0,1,2],inplace=True)
data['Initial'].replace(['Mr','Mrs','Miss','Master','Other'],[0,1,2,3,4],inplace=True)

불필요한 기능 삭제
name--> 범주형 값으로 변환할 수 없으므로 이름 기능이 필요하지 않습니다.

Age--> Age_band 기능이 있으므로 필요하지 않습니다.

Ticket--> 분류할 수 없는 임의의 문자열입니다.

Fare--> Fate_cat 기능이 있으므로 필요하지 않습니다.

Cabin--> 많은 NaN 값과 많은 승객이 여러 개의 객실을 가지고 있습니다. 따라서 이것은 쓸모없는 기능입니다.

Fare_Range--> Fare_cat 기능이 있습니다.

PassengerId--> index가 있이므로 필요하지 않음.

In [47]:
data.drop(['Name','Age','Ticket','Fare','Cabin','Fare_Range','PassengerId'],axis=1,inplace=True)
sns.heatmap(data.corr(),annot=True,cmap='RdYlGn',linewidths=0.2,annot_kws={'size':20})
#annot_kws : 글자 크기 조절 가능, fmt="f" : 데이터 표시 데이터타입
fig = plt.gcf()
fig.set_size_inches(18,15)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.show()

* Family Size는 SibSp, Parch를 통해 만든 요약변수(파생변수보단 요약에 가깝다고 생각)다 보니 상관관계가 높다.
* Fare_Cat도 PClass와 상관관가 높은 편에 속한다.
* Alone 또한 Family_Size를 통한 요약변수라 관계성이 짙은 편이다
* 상관관계가 높은 변수는 다중공산성을 조심해야 한다.

# Part3 : Predictive Modeling <br> 예측 모델링

In [48]:
# improting all the required ML packages

from sklearn.linear_model import LogisticRegression #logistic regression(로지스틱 회귀)
from sklearn import svm #support vector Machine
from sklearn.ensemble import RandomForestClassifier #Random Forest(랜덤 포레스트)
from sklearn.neighbors import KNeighborsClassifier #KNN (KNeighbor)
from sklearn.naive_bayes import GaussianNB #Naive bayes (가우시안 나이브)
from sklearn.tree import DecisionTreeClassifier #Decision Tree(의사결정 나무)
from sklearn.model_selection import train_test_split #training and testing data split
from sklearn import metrics #accuracy measure
from sklearn.metrics import confusion_matrix #for confusion matrix

In [49]:
train,test =train_test_split(data,test_size =0.3,random_state =0, stratify=data['Survived'])
# stratify : Stratified K Fold 역할과 비슷. Stratify를 Target 변수로 지정해주면 비율에 맞게 나눠준다
train_X = train[train.columns[1:]] #설명 변수들
train_Y = train[train.columns[:1]] # Survived(목표 변)

test_X = test[test.columns[1:]]
test_Y = test[test.columns[:1]]

X=data[data.columns[1:]]
Y=data['Survived']

# Radial Support Vector Machines (rbf- SVM)

In [50]:
model = svm.SVC(kernel ='rbf',C=1,gamma=0.1)
model.fit(train_X,train_Y)

prediction1=model.predict(test_X)

print('Accuracy for rbf SVM is',metrics.accuracy_score(prediction1,test_Y))

# Linear Support Vector Machine (linear-SVM)

In [51]:
model=svm.SVC(kernel='linear',C=0.1,gamma=0.1)
model.fit(train_X,train_Y)
prediction2=model.predict(test_X)
print('Accuracy for linear SVM is',metrics.accuracy_score(prediction2,test_Y))

# Logistic Regression

In [52]:
model = LogisticRegression()
model.fit(train_X,train_Y)
prediction3=model.predict(test_X)
print('The accuracy of the Logistic Regression is',metrics.accuracy_score(prediction3,test_Y))

# Decision Tree

In [53]:
model=DecisionTreeClassifier()
model.fit(train_X,train_Y)
prediction4=model.predict(test_X)
print('The accuracy of the Decision Tree is',metrics.accuracy_score(prediction4,test_Y))

# K-Nearest Neighbours(KNN)

In [54]:
model=KNeighborsClassifier() 
model.fit(train_X,train_Y)
prediction5=model.predict(test_X)
print('The accuracy of the KNN is',metrics.accuracy_score(prediction5,test_Y))

* n_neighbors 속성의 값을 변경하면 KNN 모델의 정확도가 변경됨 (기본값은 5)


In [55]:
a_index = list(range(1,11))
a=pd.Series()
x=[0,1,2,3,4,5,6,7,8,9,10]

for i in list(range(1,11)):
    model=KNeighborsClassifier(n_neighbors=i) 
    model.fit(train_X,train_Y)
    prediction=model.predict(test_X)
    a = a.append(pd.Series(metrics.accuracy_score(prediction,test_Y)))

In [56]:
plt. plot(a_index,a)
plt.xticks(x)
fig = plt.gcf()
fig.set_size_inches(12,6)
plt.show()

print('Accuracies for different values of n are:',a.values,'with the max value as ',a.values.max())

# Gaussian Naive Bayes

In [57]:
model = GaussianNB()
model.fit(train_X,train_Y)
prediction6 = model.predict(test_X)
print('The accuracy of the NaiveBayes is',metrics.accuracy_score(prediction6,test_Y))

# Random Forests

In [58]:
model=RandomForestClassifier(n_estimators=100)
model.fit(train_X,train_Y)
prediction7=model.predict(test_X)
print('The accuracy of the Random Forests is',metrics.accuracy_score(prediction7,test_Y))

# Cross Validation (교차검증)

* 모델의 정확도로 최적화된 학습 모델이라고 평가할 수 없음 때문에 교차검증 진행

* 대부분 데이터가 불균형함. 

1) The K-Fold Cross Validation : dataset를 k-subsets으로 나누는 방식으로 작동
2) data 집합을 (k=5)개의 부분으로 나눈다고 가정. 4개를 훈련에 쓰고 1개를 테스트하는 방식
3) 반복적으로 테스트 부분을 변경하고 다른 부분에 대해서도 알고리즘 훈련함으로써 프로세스를 계속 실시하면  정확도와 오류는 알고리즘의 평균 정확도를 얻기 위해 평균화됩니다

* The K-Fold Cross Validation에 관하여 찾아보길 바란다

In [59]:
from sklearn.model_selection import KFold #for K-fold cross validation
from sklearn.model_selection import cross_val_score #score evaluation
from sklearn.model_selection import cross_val_predict #prediction
kfold = KFold(n_splits=10 ,shuffle=True,random_state=22) # k=10, split the data into 10 equal parts
xyz=[]
accuracy=[]
std=[]
classifiers=['Linear Svm','Radial Svm','Logistic Regression','KNN','Decision Tree','Naive Bayes','Random Forest']
models=[svm.SVC(kernel='linear'),svm.SVC(kernel='rbf'),LogisticRegression(),KNeighborsClassifier(n_neighbors=9),DecisionTreeClassifier(),GaussianNB(),RandomForestClassifier(n_estimators=100)]
for i in models:
    model = i
    cv_result = cross_val_score(model,X,Y, cv = kfold,scoring = "accuracy")
    cv_result=cv_result
    xyz.append(cv_result.mean())
    std.append(cv_result.std())
    accuracy.append(cv_result)
new_models_dataframe2=pd.DataFrame({'CV Mean':xyz,'Std':std},index=classifiers)       
print(new_models_dataframe2)

In [60]:
plt.subplots(figsize=(12,6))
box = pd.DataFrame(accuracy,index=[classifiers])
box.T.boxplot()

In [61]:
new_models_dataframe2['CV Mean'].plot.barh(width=0.8)
plt.title('Average CV Mean Accuracy')
fig = plt.gcf()
fig.set_size_inches(8,5)
plt.show()

* 모든 예측 모델 중 Radial SVM이 가장 성능이 좋은것으로 나타남

# Confusion Matrix

In [62]:
f,ax=plt.subplots(3,3,figsize=(12,10))
y_pred = cross_val_predict(svm.SVC(kernel='rbf'),X,Y,cv=10)
sns.heatmap(confusion_matrix(Y,y_pred),ax=ax[0,0],annot=True,fmt='2.0f')
ax[0,0].set_title('Matrix for rbf-SVM')
y_pred = cross_val_predict(svm.SVC(kernel='linear'),X,Y,cv=10)
sns.heatmap(confusion_matrix(Y,y_pred),ax=ax[0,1],annot=True,fmt='2.0f')
ax[0,1].set_title('Matrix for Linear-SVM')
y_pred = cross_val_predict(KNeighborsClassifier(n_neighbors=9),X,Y,cv=10)
sns.heatmap(confusion_matrix(Y,y_pred),ax=ax[0,2],annot=True,fmt='2.0f')
ax[0,2].set_title('Matrix for KNN')
y_pred = cross_val_predict(RandomForestClassifier(n_estimators=100),X,Y,cv=10)
sns.heatmap(confusion_matrix(Y,y_pred),ax=ax[1,0],annot=True,fmt='2.0f')
ax[1,0].set_title('Matrix for Random-Forests')
y_pred = cross_val_predict(LogisticRegression(),X,Y,cv=10)
sns.heatmap(confusion_matrix(Y,y_pred),ax=ax[1,1],annot=True,fmt='2.0f')
ax[1,1].set_title('Matrix for Logistic Regression')
y_pred = cross_val_predict(DecisionTreeClassifier(),X,Y,cv=10)
sns.heatmap(confusion_matrix(Y,y_pred),ax=ax[1,2],annot=True,fmt='2.0f')
ax[1,2].set_title('Matrix for Decision Tree')
y_pred = cross_val_predict(GaussianNB(),X,Y,cv=10)
sns.heatmap(confusion_matrix(Y,y_pred),ax=ax[2,0],annot=True,fmt='2.0f')
ax[2,0].set_title('Matrix for Naive Bayes')
plt.subplots_adjust(hspace=0.2,wspace=0.2)
plt.show()

왼쪽 대각선은 각 클래스에 대해 수행된 올바른 예측의 수를 나타내고 오른쪽 대각선은 잘못된 예측의 수를 나타냅니다. rbf-SVM에 대한 첫 번째 그림을 살펴보겠습니다.

1) 올바른 예측의 수는 491(사망자의 경우) + 247(생존자의 경우)이며, 평균 CV 정확도는 (491+247)/891 = 82.8%

2) 오류--> 58명의 사망자를 생존자로 잘못 분류하고 95명을 생존자로 분류했습니다. 그래서 죽은 자를 살아남은 것으로 예측함으로써 더 많은 실수가 있음

* 모든 행렬을 살펴봄으로써, 우리는 rbf-SVM이 사망 승객을 정확하게 예측할 확률이 높지만, NaiveBayes는 생존 승객을 정확하게 예측할 확률이 더 높다고 말할 수 있다.

# Hyper Parameters 튜닝

* 머신러닝에서 사용자가 직접 튜닝함으로서 모델의 성능을 조절할 수 있다

# SVM

In [63]:
from sklearn.model_selection import GridSearchCV
C=[0.05,0.1,0.2,0.3,0.25,0.4,0.5,0.6,0.7,0.8,0.9,1]
gamma=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
kernel=['rbf','linear']
hyper={'kernel':kernel,'C':C,'gamma':gamma}
gd=GridSearchCV(estimator=svm.SVC(),param_grid=hyper,verbose=True)
# verbose : iteration시마다 수행결과 메시지 출력 여부
# 0 : 메시지 출력 ㄴ(default)
# 1 : 간단한 메세지 출력
# 2 : 하이퍼 파라미터별 메시지 출력

gd.fit(X,Y)
print(gd.best_score_)
print(gd.best_estimator_)

#SVC 머신러닝 모델의 하이퍼 파라미터는 대표적으로 C, gamma, kernel이 있다. 
#이를 딕셔너리 자료구조로 만들어 놓고 GridSearchCV로 돌린 코드

# Random Forests

In [64]:
n_estimators=range(100,1000,100)
hyper={'n_estimators':n_estimators}
gd=GridSearchCV(estimator=RandomForestClassifier(random_state=0),param_grid=hyper,verbose=True)
gd.fit(X,Y)
print(gd.best_score_)
print(gd.best_estimator_)

# RandomForestClassifier의 하이퍼 파라미터 중 n_estimators로 성능 조절

# Ensembling

* 모델의 정확도 또는 성능을 향상시키는 좋은 방법
* 다양한 단순 모델을 조합해 하나의 강력한 모델을 만드는 것<br>
1) Voting Classifier<br>
2) Bagging<br>
3) Boosting.<br>

# Voting Classifier

* Voting Classifier은 여러 다른 ML 모델의 예측을 결합한 가장 간단한 방법이다.
* Voting은 여러 모형에서 산출된 결과를 다수결에 의해 최종 결과를 선정하는 과정이다.

In [65]:
from sklearn.ensemble import VotingClassifier
ensemble_lin_rbf=VotingClassifier(estimators=[('KNN',KNeighborsClassifier(n_neighbors=10)),
                                              ('RBF',svm.SVC(probability=True,kernel='rbf',C=0.5,gamma=0.1)),
                                              ('RFor',RandomForestClassifier(n_estimators=500,random_state=0)),
                                              ('LR',LogisticRegression(C=0.05)),
                                              ('DT',DecisionTreeClassifier(random_state=0)),
                                              ('NB',GaussianNB()),
                                              ('svm',svm.SVC(kernel='linear',probability=True))
                                             ], 
                       voting='soft').fit(train_X,train_Y)
print('The accuracy for ensembled model is:',ensemble_lin_rbf.score(test_X,test_Y))
cross=cross_val_score(ensemble_lin_rbf,X,Y, cv = 10,scoring = "accuracy")
print('The cross validated score is',cross.mean())

# Bagging

Bagging은 분산이 높은 모델에서 잘 작동되는 모델이라 한다. Bagging은 주어진 자료에서 여러개의 붓스트랩 자료를 생성하고 각 붓스트랩 자료에서 예측모형을 만든 후 결합하여 최종 예측 모델을 만든다.

* 붓스트랩 : 주어진 자료에서 동일한 크기의 표본을 랜덤 복원추출로 뽑은 자료 의미

## Bagged KNN

In [66]:
from sklearn.ensemble import BaggingClassifier
model=BaggingClassifier(base_estimator=KNeighborsClassifier(n_neighbors=3),random_state=0,n_estimators=700)
model.fit(train_X,train_Y)
prediction=model.predict(test_X)
print('The accuracy for bagged KNN is:',metrics.accuracy_score(prediction,test_Y))
result=cross_val_score(model,X,Y,cv=10,scoring='accuracy')
print('The cross validated score for bagged KNN is:',result.mean())

## Bagged Decision Tree

In [67]:
model=BaggingClassifier(base_estimator=DecisionTreeClassifier(),random_state=0,n_estimators=100)
model.fit(train_X,train_Y)
prediction=model.predict(test_X)
print('The accuracy for bagged Decision Tree is:',metrics.accuracy_score(prediction,test_Y))
result=cross_val_score(model,X,Y,cv=10,scoring='accuracy')
print('The cross validated score for bagged Decision Tree is:',result.mean())

# Boosting

Boosting하면 가장 먼저 생각나는 것은 XGBoost인데, 이 외에도 여러가지가 있었다. AdaBoost, GridientBoost가 있지만 XGBoost가 가장 좋은 모델이라고 한다.

* Boosting은 분류기의 순차 학습을 사용하는 앙상블 기술로 약한 모델을 단계별로 개선한 것이다

[과정]
> * 모델은 먼저 전체 dataset에 대해 훈련하고 몇가지 인스턴스를 맞히고 몇가지 인스턴스를 틀리게 됩니다
> * 다음 반복에는 학습 때 잘못 예측된 인스턴스에 더 초점을 맞추거나 더 많은 가중치를 부여하여 잘못된 인스턴스를 정확하게 예측하고자 하는 것
> * 계속 위의 프로세스를 반복하여 정확도에 대한 한계에 도달할때까지 새로운 분류기가 모델에 추가됨

## AdaBoost(Adaptive Boosting)

약한 학습자 또는 추정자는 결정 트리입니다. 그러나 default base_estimator를 원하는 알고리즘으로 변경할 수 있습니다.

In [68]:
from sklearn.ensemble import AdaBoostClassifier
ada=AdaBoostClassifier(n_estimators=200,random_state=0,learning_rate=0.1)
result=cross_val_score(ada,X,Y,cv=10,scoring='accuracy')
print('The cross validated score for AdaBoost is:',result.mean())

## Stochastic Gradient Boosting

약한 학습자는 의사결정 나무이다.

In [69]:
from sklearn.ensemble import GradientBoostingClassifier
grad=GradientBoostingClassifier(n_estimators=500,random_state=0,learning_rate=0.1)
result=cross_val_score(grad,X,Y,cv=10,scoring='accuracy')
print('The cross validated score for Gradient Boosting is:',result.mean())

## XGBoost

In [70]:
import xgboost as xg
xgboost=xg.XGBClassifier(n_estimators=900,learning_rate=0.1)
result=cross_val_score(xgboost,X,Y,cv=10,scoring='accuracy')
print('The cross validated score for XGBoost is:',result.mean())

## Hyper-Parameter Tuning for AdaBoost

In [71]:
n_estimators=list(range(100,1100,100))
learn_rate=[0.05,0.1,0.2,0.3,0.25,0.4,0.5,0.6,0.7,0.8,0.9,1]
hyper={'n_estimators':n_estimators,'learning_rate':learn_rate}
gd=GridSearchCV(estimator=AdaBoostClassifier(),param_grid=hyper,verbose=True)
gd.fit(X,Y)
print(gd.best_score_)
print(gd.best_estimator_)

## Confusion Matrix for the Best Model

In [72]:
ada=AdaBoostClassifier(n_estimators=200,random_state=0,learning_rate=0.05)
result=cross_val_predict(ada,X,Y,cv=10)
sns.heatmap(confusion_matrix(Y,result),cmap='winter',annot=True,fmt='2.0f')
plt.show()

## Feature Importance

In [73]:
f,ax=plt.subplots(2,2,figsize=(15,12))
model=RandomForestClassifier(n_estimators=500,random_state=0)
model.fit(X,Y)
pd.Series(model.feature_importances_,X.columns).sort_values(ascending=True).plot.barh(width=0.8,ax=ax[0,0])
ax[0,0].set_title('Feature Importance in Random Forests')
model=AdaBoostClassifier(n_estimators=200,learning_rate=0.05,random_state=0)
model.fit(X,Y)
pd.Series(model.feature_importances_,X.columns).sort_values(ascending=True).plot.barh(width=0.8,ax=ax[0,1],color='#ddff11')
ax[0,1].set_title('Feature Importance in AdaBoost')
model=GradientBoostingClassifier(n_estimators=500,learning_rate=0.1,random_state=0)
model.fit(X,Y)
pd.Series(model.feature_importances_,X.columns).sort_values(ascending=True).plot.barh(width=0.8,ax=ax[1,0],cmap='RdYlGn_r')
ax[1,0].set_title('Feature Importance in Gradient Boosting')
model=xg.XGBClassifier(n_estimators=900,learning_rate=0.1)
model.fit(X,Y)
pd.Series(model.feature_importances_,X.columns).sort_values(ascending=True).plot.barh(width=0.8,ax=ax[1,1],color='#FD0F00')
ax[1,1].set_title('Feature Importance in XgBoost')
plt.show()